In [1]:
import torch as t
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM
import argparse
import pandas as pd
from tqdm import tqdm
import os
import configparser
import glob

# config = configparser.ConfigParser()
# config.read('config.ini')

# class Hook:
#     def __init__(self):
#         self.out = None

#     def __call__(self, module, module_inputs, module_outputs):
#         self.out, _ = module_outputs

# def load_model(model_family: str, model_size: str, model_type: str, device: str):
#     model_path = os.path.join(config[model_family]['weights_directory'], 
#                               config[model_family][f'{model_size}_{model_type}_subdir'])
    
#     try:
#         if model_family == 'Llama2':
#             tokenizer = LlamaTokenizer.from_pretrained(str(model_path))
#             model = LlamaForCausalLM.from_pretrained(str(model_path))
#             tokenizer.bos_token = '<s>'
#         else:
#             tokenizer = AutoTokenizer.from_pretrained(str(model_path))
#             model = AutoModelForCausalLM.from_pretrained(str(model_path))
#         if model_family == "Gemma2": # Gemma2 requires bfloat16 precision which is only available on new GPUs
#             model = model.to(t.bfloat16) # Convert the model to bfloat16 precision
#         else:
#             model = model.half()  # storing model in float32 precision -> conversion to float16
#         return tokenizer, model.to(device)
#     except Exception as e:
#         print(f"Error loading model: {e}")
#         raise

# def load_statements(dataset_name):
#     """
#     Load statements from csv file, return list of strings.
#     """
#     dataset = pd.read_csv(f"datasets/open_ended_truth/{dataset_name}.csv")
#     statements = dataset['question_with_answer'].tolist()
#     return statements

# def get_acts(statements, tokenizer, model, layers, device):
#     """
#     Get given layer activations for the statements. 
#     Return dictionary of stacked activations.
#     """
#     # attach hooks
#     hooks, handles = [], []
#     for layer in layers:
#         hook = Hook()
#         handle = model.model.layers[layer].register_forward_hook(hook)
#         hooks.append(hook), handles.append(handle)
    
#     # get activations
#     acts = {layer : [] for layer in layers}
#     for statement in tqdm(statements):
#         input_ids = tokenizer.encode(statement, return_tensors="pt").to(device)
#         model(input_ids)
#         for layer, hook in zip(layers, hooks):
#             acts[layer].append(hook.out[0, -1])
    
#     for layer, act in acts.items():
#         acts[layer] = t.stack(act).float()
    
#     # remove hooks
#     for handle in handles:
#         handle.remove()
    
#     return acts

# if __name__ == "__main__":
#     """
#     read statements from dataset, record activations in given layers, and save to specified files
#     """
#     parser = argparse.ArgumentParser(description="Generate activations for statements in a dataset")
#     parser.add_argument("--model_family", default="Llama3", help="Model family to use. Options are Llama2, Llama3, Gemma, Gemma2 or Mistral.")
#     parser.add_argument("--model_size", default="8B",
#                         help="Size of the model to use. Options for Llama3 are 8B or 70B")
#     parser.add_argument("--model_type", default="base", help="Whether to choose base or chat model. Options are base or chat.")
#     parser.add_argument("--layers", nargs='+', 
#                         help="Layers to save embeddings from.")
#     parser.add_argument("--datasets", nargs='+',
#                         help="Names of datasets, without .csv extension")
#     parser.add_argument("--output_dir", default="acts",
#                         help="Directory to save activations to.")
#     parser.add_argument("--device", default="cpu")
#     args = parser.parse_args()

    # datasets = args.datasets
    # if datasets == ['all_topic_specific']:
    #     datasets = ['cities', 'sp_en_trans', 'inventors', 'animal_class', 'element_symb', 'facts',
    #                 'neg_cities', 'neg_sp_en_trans', 'neg_inventors', 'neg_animal_class', 'neg_element_symb', 'neg_facts',
    #                 'cities_conj', 'sp_en_trans_conj', 'inventors_conj', 'animal_class_conj', 'element_symb_conj', 'facts_conj',
    #                 'cities_disj', 'sp_en_trans_disj', 'inventors_disj', 'animal_class_disj', 'element_symb_disj', 'facts_disj',
    #                 'larger_than', 'smaller_than', "cities_de", "neg_cities_de", "sp_en_trans_de", "neg_sp_en_trans_de", "inventors_de", "neg_inventors_de", "animal_class_de",
    #               "neg_animal_class_de", "element_symb_de", "neg_element_symb_de", "facts_de", "neg_facts_de"]
    # if datasets == ['all']:
    #     datasets = []
    #     for file_path in glob.glob('datasets/**/*.csv', recursive=True):
    #         dataset_name = os.path.relpath(file_path, 'datasets').replace('.csv', '')
    #         datasets.append(dataset_name)

    # t.set_grad_enabled(False)
    # tokenizer, model = load_model(args.model_family, args.model_size, args.model_type, args.device)
    
    # for dataset in datasets:
    #     statements = load_statements(dataset)
    #     layers = [int(layer) for layer in args.layers]
    #     if layers == [-1]:
    #         layers = list(range(len(model.model.layers)))
    #     save_dir = f"{args.output_dir}/{args.model_family}/{args.model_size}/{args.model_type}/{dataset}/"
    #     if not os.path.exists(save_dir):
    #         os.makedirs(save_dir)

    #     for idx in range(0, len(statements), 25):
    #         acts = get_acts(statements[idx:idx + 25], tokenizer, model, layers, args.device)
    #         for layer, act in acts.items():
    #                 t.save(act, f"{save_dir}/layer_{layer}_{idx}.pt")

/workspace/Truth_is_Universal/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
class Hook:
    def __init__(self):
        self.out = None

    def __call__(self, module, module_inputs, module_outputs):
        self.out, _ = module_outputs

def load_model(model_family: str, model_size: str, model_type: str, device: str):
    # model_path = os.path.join(config[model_family]['weights_directory'], 
    #                           config[model_family][f'{model_size}_{model_type}_subdir'])
    
    model_path = "../../../../models/llama3_8b_chat_hf"
    
    try:
        if model_family == 'Llama2':
            tokenizer = LlamaTokenizer.from_pretrained(str(model_path))
            model = LlamaForCausalLM.from_pretrained(str(model_path))
            tokenizer.bos_token = '<s>'
        else:
            tokenizer = AutoTokenizer.from_pretrained(str(model_path))
            model = AutoModelForCausalLM.from_pretrained(str(model_path))
        if model_family == "Gemma2": # Gemma2 requires bfloat16 precision which is only available on new GPUs
            model = model.to(t.bfloat16) # Convert the model to bfloat16 precision
        else:
            model = model.half()  # storing model in float32 precision -> conversion to float16
        return tokenizer, model.to(device)
    except Exception as e:
        print(f"Error loading model: {e}")
        raise

def load_statements(dataset_name):
    """
    Load statements from csv file, return list of strings.
    """
    dataset = pd.read_csv(f"../../../datasets/open_ended_truth/{dataset_name}.csv")
    statements = dataset['question_with_answer'].tolist()
    return statements

def get_acts(statements, tokenizer, model, layers, device):
    """
    Get given layer activations for the statements. 
    Return dictionary of stacked activations.
    """
    # attach hooks
    hooks, handles = [], []
    for layer in layers:
        hook = Hook()
        handle = model.model.layers[layer].register_forward_hook(hook)
        hooks.append(hook), handles.append(handle)
    
    # get activations
    acts = {layer : [] for layer in layers}
    for statement in tqdm(statements):
        input_ids = tokenizer.encode(statement, return_tensors="pt").to(device)
        model(input_ids)
        for layer, hook in zip(layers, hooks):
            acts[layer].append(hook.out[0, -1])
    
    for layer, act in acts.items():
        acts[layer] = t.stack(act).float()
    
    # remove hooks
    for handle in handles:
        handle.remove()
    
    return acts

In [17]:
datasets = [
    "cities_questions",
    "element_symb_questions",
    "inventors_questions",
]

In [20]:
model_family = "Llama3"
model_size = "8B"
model_type = "chat"
device = "cuda:0"
layers = [12]
output_dir = "../../../acts/open_ended_truth_acts"

In [21]:
t.set_grad_enabled(False)
tokenizer, model = load_model(model_family, model_size, model_type, device)

for dataset in datasets:
    statements = load_statements(dataset)
    layers = [int(layer) for layer in layers]
    if layers == [-1]:
        layers = list(range(len(model.model.layers)))
    save_dir = f"{output_dir}/{model_family}/{model_size}/{model_type}/{dataset}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for idx in range(0, len(statements), 25):
        acts = get_acts(statements[idx:idx + 25], tokenizer, model, layers, device)
        for layer, act in acts.items():
                t.save(act, f"{save_dir}/layer_{layer}_{idx}.pt")

100%|██████████| 6/6 [00:00<00:00, 42.50it/s]
